## CONTEXTUAL FEATURE SELECTION WITH CONDITIONAL STOCHASTIC GATES

In [1]:
from sklearn.datasets import fetch_california_housing
import pandas as pd
import numpy as np

# Load dataset
housing = fetch_california_housing()
X, y = housing.data, housing.target

# Reduce to maximum 1000 rows
# X = X[:1000, :]
# y = y[:1000]

# Original column names
column_names = housing.feature_names
feature_names = np.array(column_names)
# Add noise columns
np.random.seed(42)  # For reproducibility

# Gaussian noise
gaussian_noise = np.random.normal(0, 1, size=X.shape[0])

# Uniform noise
uniform_noise = np.random.uniform(-1, 1, size=X.shape[0])

# Cosine function
cosine_values = np.cos(np.linspace(0, 10, X.shape[0]))

# Create a DataFrame from X
df = pd.DataFrame(X, columns=column_names)

# Add the noise columns to DataFrame
df['Gaussian_Noise'] = gaussian_noise
df['Uniform_Noise'] = uniform_noise
df['Cosine_Values'] = cosine_values

# Shuffle column locations
np.random.seed(42)  # Ensure reproducibility for column shuffling
shuffled_columns = np.random.permutation(df.columns)
df = df[shuffled_columns]

# Now, df is a DataFrame with shuffled columns and includes the noise features
# You can view the DataFrame as follows:
print(df.head())

# Convert target to a DataFrame and concatenate with features for a complete view
y_df = pd.DataFrame(y, columns=['Target'])
df_full = pd.concat([df, y_df], axis=1)

# If you wish to proceed with splitting and scaling:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset

# Splitting the data (assuming you want to keep DataFrame structure for X)
X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=42)

# Scaling features
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns)
X_test_scaled = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns)

# X_train_scaled and X_test_scaled are now DataFrames with scaled features and retained column names.
# convert them to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled.values, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_scaled.values, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)
y_train_tensor = y_train_tensor.view(-1, 1)
y_test_tensor = y_test_tensor.view(-1, 1)

# Create TensorDataset
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# new column names
feature_names = np.array(X_train_scaled.columns)
# feature_names

   AveOccup  MedInc  Uniform_Noise  Cosine_Values  AveRooms  HouseAge  \
0  2.555556  8.3252       0.143745       1.000000  6.984127      41.0   
1  2.109842  8.3014      -0.515390       1.000000  6.238137      21.0   
2  2.802260  7.2574       0.677635       1.000000  8.288136      52.0   
3  2.547945  5.6431      -0.038881       0.999999  5.817352      52.0   
4  2.181467  3.8462       0.285882       0.999998  6.281853      52.0   

   Gaussian_Noise  Population  Longitude  AveBedrms  Latitude  
0        0.496714       322.0    -122.23   1.023810     37.88  
1       -0.138264      2401.0    -122.22   0.971880     37.86  
2        0.647689       496.0    -122.24   1.073446     37.85  
3        1.523030       558.0    -122.25   1.073059     37.85  
4       -0.234153       565.0    -122.25   1.081081     37.85  


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Normal, Bernoulli


##

 Define the Hypernetwork

The hypernetwork takes contextual variables as input and outputs parameters for the Bernoulli distributions of the stochastic gates.

In [3]:
class HyperNetwork(nn.Module):
    def __init__(self, context_dim, feature_dim):
        super(HyperNetwork, self).__init__()
        # Define a simple feed-forward network as an example
        self.network = nn.Sequential(
            nn.Linear(context_dim, 64),
            nn.ReLU(),
            nn.Linear(64, feature_dim)
            # No sigmoid activation here because we do not want to restrict the output between 0 and 1 yet
        )

    def forward(self, context):
        # Outputs the parameters for the Bernoulli distributions
        return self.network(context)


def stochastic_gates(mu, sigma=1.0):
    """
    Applies Gaussian-based continuous relaxation to the Bernoulli variables.
    :param mu: Mean parameters of the Bernoulli distributions.
    :param sigma: Standard deviation for the Gaussian noise.
    :return: Relaxed gates after applying noise and clipping.
    """
    # Add Gaussian noise to the parameters
    epsilon = torch.randn_like(mu) * sigma
    # Apply the continuous relaxation
    gates = torch.clamp(mu + epsilon, 0, 1)
    return gates

# Fixed sigma value as per the process described
fixed_sigma = 1.0  # or any other value specified in the article or experiment setup



In [4]:

# class HyperNetwork(nn.Module):
#     def __init__(self, context_dim, feature_dim):
#         super(HyperNetwork, self).__init__()
#         self.network = nn.Sequential(
#             nn.Linear(context_dim, 64),  # Adjust sizes as necessary
#             nn.ReLU(),
#             nn.Linear(64, feature_dim),
#             nn.Sigmoid()  # Ensures output is between 0 and 1
#         )

#     def forward(self, context):
#         return self.network(context)


Define the Prediction Network

The prediction network maps the selected features to the response variable. It takes both the original features and the stochastic gates as inputs.

In [5]:
class PredictionNetwork(nn.Module):
    def __init__(self, feature_dim):
        super(PredictionNetwork, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(feature_dim, 64),  # Adjust sizes as necessary
            nn.ReLU(),
            nn.Linear(64, 1)  # Assuming a single continuous outcome
        )

    def forward(self, features):
        return self.network(features)


In [6]:
# def stochastic_gates(bernoulli_params, device='cpu'):
#     sigma = 1.0  # Hyperparameter for Gaussian noise
#     epsilon = Normal(0, sigma).sample(bernoulli_params.shape).to(device)
#     gates = torch.sigmoid(bernoulli_params + epsilon)  # Apply the sigmoid to approximate the Bernoulli distribution
#     return gates


5. Training Loop


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim

# Loss function
loss_fn = nn.MSELoss()

# Assuming the definitions of HyperNetwork, PredictionNetwork, stochastic_gates, and the dataset preparation code are as previously defined

# Define a function to dynamically adjust the size of the prediction network based on context dimension
def adjust_prediction_network(input_feature_dim):
    return PredictionNetwork(input_feature_dim)

# list_contexts = [2, 3, 4, 5, 6, 7, 8, 9, 10]  # List of context dimensions to try

# Initialize dictionary to store feature combinations and their best validation loss
# feature_combinations = {}

# best_val_loss = float('inf')
# explanatory_feature_names = feature_names[context_dim:]
# print(f'\nTraining with first {context_dim} features as context\n' + '-'*50)

# Reinitialize networks for each context dimension
hypernetwork = HyperNetwork(context_dim, X_train_tensor.shape[1] - context_dim)  # Adjust the input size of hypernetwork based on context_dim
prediction_network = adjust_prediction_network(X_train_tensor.shape[1] - context_dim)  # Adjust the input size of prediction network based on remaining features

# Reinitialize the optimizer
optimizer = optim.Adam(list(hypernetwork.parameters()) + list(prediction_network.parameters()), lr=0.001)

num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Training step
    contexts = X_train_tensor[:, :context_dim]
    features = X_train_tensor[:, context_dim:]
    
    bernoulli_params = hypernetwork(contexts)
    gates = stochastic_gates(bernoulli_params, sigma=fixed_sigma)    
    # selected_features = features * gates
    predictions = prediction_network(selected_features)
    
    loss = loss_fn(predictions, y_train_tensor)
    loss.backward()
    optimizer.step()

    # Validation step
    with torch.no_grad():
        contexts_val = X_test_tensor[:, :context_dim]
        features_val = X_test_tensor[:, context_dim:]
        
        bernoulli_params_val = hypernetwork(contexts_val)
        gates_val = stochastic_gates(bernoulli_params, sigma=fixed_sigma)    
        selected_features_val = features_val * gates_val
        predictions_val = prediction_network(selected_features_val)
        
        val_loss = loss_fn(predictions_val, y_test_tensor)

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')
    
    # Store feature combinations based on the top N important features for each epoch
    N = 5  # Adjust based on the desired top N features
    avg_bernoulli_params = bernoulli_params_val.mean(dim=0)
    top_features_indices = torch.argsort(avg_bernoulli_params, descending=True)[:N]
    top_features_tuple = tuple(top_features_indices.cpu().numpy())
    
    feature_names_tuple = tuple(explanatory_feature_names[top_features_indices])
    
    if feature_names_tuple in feature_combinations:
        if feature_combinations[feature_names_tuple]['val_loss'] > val_loss.item():
            feature_combinations[feature_names_tuple]['val_loss'] = val_loss.item()
    else:
        feature_combinations[feature_names_tuple] = {'val_loss': val_loss.item(), 'avg_params': avg_bernoulli_params[top_features_indices].cpu().numpy()}

    if epoch % 10 == 0 or epoch == num_epochs - 1:
        print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

# Display the best feature combination
best_combination = min(feature_combinations.items(), key=lambda x: x[1]['val_loss'])
print("\nBest feature combination (names):", best_combination[0])
print("Validation loss for best combination:", best_combination[1]['val_loss'])
print("Average Bernoulli parameters for best combination:", best_combination[1]['avg_params'])

NameError: name 'context_dim' is not defined

In [ ]:

# num_epochs = 100
# for epoch in range(num_epochs):
#     optimizer.zero_grad()
    
#     # Training step
#     contexts = X_train_tensor[:, :context_dim]
#     features = X_train_tensor[:, context_dim:]
    
#     bernoulli_params = hypernetwork(contexts)
#     gates = stochastic_gates(bernoulli_params)
#     selected_features = features * gates
#     predictions = prediction_network(selected_features)
    
#     loss = loss_fn(predictions, y_train_tensor)
#     loss.backward()
#     optimizer.step()

#     # Validation step
#     with torch.no_grad():
#         contexts_val = X_test_tensor[:, :context_dim]
#         features_val = X_test_tensor[:, context_dim:]
        
#         bernoulli_params_val = hypernetwork(contexts_val)
#         gates_val = stochastic_gates(bernoulli_params_val)
#         selected_features_val = features_val * gates_val
#         predictions_val = prediction_network(selected_features_val)
        
#         val_loss = loss_fn(predictions_val, y_test_tensor)

#     if epoch % 10 == 0:
#         print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')


To present the best feature combinations found by the model, we can analyze the gates (stochastic gates) produced by the hypernetwork for each feature across the validation set. The gates closest to 1 indicate features that are most relevant for the prediction according to the model. However, since the gates are produced in a stochastic manner, a straightforward approach is to use the mean of the Bernoulli parameters (π) output by the hypernetwork as an approximation of feature importance. This way, we can determine which features are consistently considered important across different contexts.

Below is the code to achieve this, including a modification to compute the average Bernoulli parameters across the validation set and then display the best feature combinations based on these averages:

In [10]:
list_contexts = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]  # List of context dimensions to try
for context_dim in list_contexts:
    # Assume first 3 features are "contextual" for demonstration purposes
    num_epochs = 100
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Training step
        contexts = X_train_tensor[:, :context_dim]
        features = X_train_tensor[:, context_dim:]
        
        bernoulli_params = hypernetwork(contexts)
        gates = stochastic_gates(bernoulli_params)
        selected_features = features * gates
        predictions = prediction_network(selected_features)
        
        loss = loss_fn(predictions, y_train_tensor)
        loss.backward()
        optimizer.step()

        # Validation step
        with torch.no_grad():
            contexts_val = X_test_tensor[:, :context_dim]
            features_val = X_test_tensor[:, context_dim:]
            
            bernoulli_params_val = hypernetwork(contexts_val)
            gates_val = stochastic_gates(bernoulli_params_val)
            selected_features_val = features_val * gates_val
            predictions_val = prediction_network(selected_features_val)
            
            val_loss = loss_fn(predictions_val, y_test_tensor)

        if epoch % 10 == 0:
            print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')
    feature_dim = X_train_tensor.shape[1] - context_dim

    # Update Hypernetwork and PredictionNetwork definitions if needed

    # Initialize models
    hypernetwork = HyperNetwork(context_dim, feature_dim)
    prediction_network = PredictionNetwork(feature_dim)

    # Optimizer and loss function
    optimizer = optim.Adam(list(hypernetwork.parameters()) + list(prediction_network.parameters()), lr=0.001)
    loss_fn = nn.MSELoss()

    hypernetwork = HyperNetwork(context_dim=context_dim, feature_dim=X_train_tensor.shape[1])


    # Initialize dictionary to store feature combinations and their best validation loss
    feature_combinations = {}

    num_epochs = 100
    best_val_loss = float('inf')

    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Training step
        contexts = X_train_tensor[:, :context_dim]
        features = X_train_tensor[:, context_dim:]
        
        bernoulli_params = hypernetwork(contexts)
        gates = stochastic_gates(bernoulli_params)
        selected_features = features * gates
        predictions = prediction_network(selected_features)
        
        loss = loss_fn(predictions, y_train_tensor)
        loss.backward()
        optimizer.step()

        # Validation step
        with torch.no_grad():
            contexts_val = X_test_tensor[:, :context_dim]
            features_val = X_test_tensor[:, context_dim:]
            
            bernoulli_params_val = hypernetwork(contexts_val)
            gates_val = stochastic_gates(bernoulli_params_val)
            selected_features_val = features_val * gates_val
            predictions_val = prediction_network(selected_features_val)
            
            val_loss = loss_fn(predictions_val, y_test_tensor)


            # Compute average Bernoulli parameters for the validation set
        with torch.no_grad():
            contexts_val = X_test_tensor[:, :context_dim]
            avg_bernoulli_params = hypernetwork(contexts_val).mean(dim=0)

        # Sort features based on average Bernoulli parameters
        sorted_features = torch.argsort(avg_bernoulli_params, descending=True)
        sorted_params = torch.sort(avg_bernoulli_params, descending=True).values

        # Print the sorted features and their corresponding Bernoulli parameters
        print("Feature Importance based on average Bernoulli parameters:")
        for i, (feature_index, importance) in enumerate(zip(sorted_features, sorted_params)):
            print(f"Feature {feature_index + context_dim} (Original Index): Importance {importance.item()}")

        # Optionally, select top N features to consider as "best" feature combinations
        N = 5  # Change N based on how many top features you want to consider
        top_features = sorted_features[:N]
        print(f"\nTop {N} Most Important Features (Adjusted Index): {top_features + context_dim}")


        # Store feature combinations based on the top N important features for each epoch
        N = 5  # Change based on desired top N features
        avg_bernoulli_params = bernoulli_params_val.mean(dim=0)
        top_features_indices = torch.argsort(avg_bernoulli_params, descending=True)[:N]
        top_features_tuple = tuple(top_features_indices.cpu().numpy())
        
        if top_features_tuple in feature_combinations:
            if feature_combinations[top_features_tuple]['val_loss'] > val_loss.item():
                feature_combinations[top_features_tuple]['val_loss'] = val_loss.item()
        else:
            feature_combinations[top_features_tuple] = {'val_loss': val_loss.item(), 'avg_params': avg_bernoulli_params[top_features_indices].cpu().numpy()}

        if epoch % 10 == 0 or epoch == num_epochs - 1:
            print(f'Epoch {epoch+1}, Training Loss: {loss.item()}, Validation Loss: {val_loss.item()}')

    # Display the best feature combination
    best_combination = min(feature_combinations.items(), key=lambda x: x[1]['val_loss'])
    print("\nBest feature combination (indices):", best_combination[0])
    print("Validation loss for best combination:", best_combination[1]['val_loss'])
    print("Average Bernoulli parameters for best combination:", best_combination[1]['avg_params'])

RuntimeError: The size of tensor a (10) must match the size of tensor b (11) at non-singleton dimension 1